In [2]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train = datasets.EMNIST(root='./data', 
                        split='balanced', 
                        train=True,
                        download=True,
                        transform=transform
                        )

test = datasets.EMNIST(root='./data', 
                        split='balanced', 
                        train=False,
                        download=True,
                        transform=transform
                        )

print('number of training data and test data: '+ str(len(train)) + ', ' + str(len(test)))
print('classes in training data: ')
print(train.classes)

number of training data and test data: 112800, 18800
classes in training data: 
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r', 't']
